# Aula 10 - Recomendação baseada em sessão - exemplos

In [4]:
import pandas as pd
import numpy as np

### Importar base de dados (executar somente se não tiver a base 2019-Oct-sample.csv. Caso contrário, pular para etapa de leitura desse arquivo).

In [5]:
# Fazer download no link: https://www.kaggle.com/datasets/mkechinov/ecommerce-behavior-data-from-multi-category-store?resource=download&select=2019-Oct.csv
#!tar -xvzf 2019-Oct.csv.zip

x 2019-Nov.csv
x 2019-Oct.csv


In [6]:
data = pd.read_csv('./2019-Oct.csv')
data.head()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2019-10-01 00:00:00 UTC,view,44600062,2103807459595387724,NaN,shiseido,35.79,541312140,72d76fde-8bb3-4e00-8c23-a032dfed738c
1,2019-10-01 00:00:00 UTC,view,3900821,2053013552326770905,appliances.environment.water_heater,aqua,33.20,554748717,9333dfbd-b87a-4708-9857-6336556b0fcc
2,2019-10-01 00:00:01 UTC,view,17200506,2053013559792632471,furniture.living_room.sofa,NaN,543.10,519107250,566511c2-e2e3-422b-b695-cf8e6e792ca8
3,2019-10-01 00:00:01 UTC,view,1307067,2053013558920217191,computers.notebook,lenovo,251.74,550050854,7c90fc70-0e80-4590-96f3-13c02c18c713
4,2019-10-01 00:00:04 UTC,view,1004237,2053013555631882655,electronics.smartphone,apple,1081.98,535871217,c6bd7419-2748-4c56-95b4-8cec9ff8b80d


In [7]:
# drop NaN values in specific columns
data = data.dropna(subset=["category_code", "brand", "user_session", "product_id"])

# keep only relevant columns in our dataset
data = data[["event_time", "event_type", "product_id", "category_code", "brand", "user_session"]]
data.head()

,event_time,event_type,product_id,category_code,brand,user_session
1,2019-10-01 00:00:00 UTC,view,3900821,appliances.environment.water_heater,aqua,9333dfbd-b87a-4708-9857-6336556b0fcc
3,2019-10-01 00:00:01 UTC,view,1307067,computers.notebook,lenovo,7c90fc70-0e80-4590-96f3-13c02c18c713
4,2019-10-01 00:00:04 UTC,view,1004237,electronics.smartphone,apple,c6bd7419-2748-4c56-95b4-8cec9ff8b80d
5,2019-10-01 00:00:05 UTC,view,1480613,computers.desktop,pulser,0d0d91c2-c9c2-4e81-90a5-86594dec0db9
8,2019-10-01 00:00:10 UTC,view,28719074,apparel.shoes.keds,baden,ac1cd4e5-a3ce-4224-a2d7-ff660a105880


In [8]:
data.sort_values(by=['user_session', 'event_time'], inplace=True, ignore_index=True)
data.head()

,event_time,event_type,product_id,category_code,brand,user_session
0,2019-10-31 06:23:12 UTC,view,1005115,electronics.smartphone,apple,00000056-a206-40dd-b174-a072550fa38c
1,2019-10-31 06:23:52 UTC,view,1005105,electronics.smartphone,apple,00000056-a206-40dd-b174-a072550fa38c
2,2019-10-31 06:25:30 UTC,view,1005105,electronics.smartphone,apple,00000056-a206-40dd-b174-a072550fa38c
3,2019-10-31 06:26:58 UTC,view,1004858,electronics.smartphone,samsung,00000056-a206-40dd-b174-a072550fa38c
4,2019-10-31 06:28:21 UTC,view,1005104,electronics.smartphone,apple,00000056-a206-40dd-b174-a072550fa38c


In [9]:
# select where to split the data
split_at = 2000000

#  make sure the split doesn't cut off session data
while data["user_session"].iloc[split_at-1] == data["user_session"].iloc[split_at]:
    split_at += 1
    
# perform the split
split_range = list(range(0, split_at))
subset = data.iloc[split_range]
subset.shape

(2000006, 6)

In [10]:
subset.to_csv('2019-Oct-sample.csv', index=False, header=['event_time', 'event_type', 'product_id', 'category_code', 'brand', 'user_session'], sep=',')


### Leitura do arquivo 2019-Oct-sample.csv

In [11]:
subset = pd.read_csv('./2019-Oct-sample.csv')
subset.head()

,event_time,event_type,product_id,category_code,brand,user_session
0,2019-10-31 06:23:12 UTC,view,1005115,electronics.smartphone,apple,00000056-a206-40dd-b174-a072550fa38c
1,2019-10-31 06:23:52 UTC,view,1005105,electronics.smartphone,apple,00000056-a206-40dd-b174-a072550fa38c
2,2019-10-31 06:25:30 UTC,view,1005105,electronics.smartphone,apple,00000056-a206-40dd-b174-a072550fa38c
3,2019-10-31 06:26:58 UTC,view,1004858,electronics.smartphone,samsung,00000056-a206-40dd-b174-a072550fa38c
4,2019-10-31 06:28:21 UTC,view,1005104,electronics.smartphone,apple,00000056-a206-40dd-b174-a072550fa38c


In [12]:
map_items = {item: idx for idx, item in enumerate(subset.product_id.unique())}
map_sessions = {item: idx for idx, item in enumerate(subset.user_session.unique())}
subset['itemId'] = subset['product_id'].map(map_items)
subset['sessionId'] = subset['user_session'].map(map_sessions)
subset.head()

,event_time,event_type,product_id,category_code,brand,user_session,itemId,sessionId
0,2019-10-31 06:23:12 UTC,view,1005115,electronics.smartphone,apple,00000056-a206-40dd-b174-a072550fa38c,0,0
1,2019-10-31 06:23:52 UTC,view,1005105,electronics.smartphone,apple,00000056-a206-40dd-b174-a072550fa38c,1,0
2,2019-10-31 06:25:30 UTC,view,1005105,electronics.smartphone,apple,00000056-a206-40dd-b174-a072550fa38c,1,0
3,2019-10-31 06:26:58 UTC,view,1004858,electronics.smartphone,samsung,00000056-a206-40dd-b174-a072550fa38c,2,0
4,2019-10-31 06:28:21 UTC,view,1005104,electronics.smartphone,apple,00000056-a206-40dd-b174-a072550fa38c,3,0


In [13]:
n_items = subset['itemId'].max()+1
print('No. items: ', n_items)
n_sessions = subset['sessionId'].max()+1
print('No. sessions: ', n_sessions)

No. items:  42581
No. sessions:  483508


In [14]:
# create a dataset
# remove sessions with less than 2 items
def create_data(df):
    df.sort_values(by=['sessionId', 'event_time'], inplace=True, ignore_index=True)
    sessions, session = [], []
    for index, value in df.iterrows():
        if index != 0:
            if value["sessionId"] == df.at[index-1, "sessionId"]:
                if value["event_type"] == 'view':
                    session.append(value["itemId"])
            else:
                if len(session) > 1:
                    sessions.append((df.at[index-1, "sessionId"], session))
                session = [value["itemId"]]
        else:
            session.append(value["itemId"])
    return sessions

In [15]:
sessions = create_data(subset)

In [16]:
print('No. sessions: ', len(sessions))
print('Session 1:', sessions[1])
subset.loc[subset.sessionId==1]

No. sessions:  296914
Session 1: (np.int64(1), [6, 7, 8, 9, 10, 11, 12, 9, 13, 9, 0, 14, 1, 15, 16, 17])


,event_time,event_type,product_id,category_code,brand,user_session,itemId,sessionId
7,2019-10-06 11:24:45 UTC,view,1004768,electronics.smartphone,samsung,00000083-8816-4d58-a9b8-f52f54186edc,6,1
8,2019-10-06 11:25:54 UTC,view,1005098,electronics.smartphone,samsung,00000083-8816-4d58-a9b8-f52f54186edc,7,1
9,2019-10-06 11:25:59 UTC,view,1005073,electronics.smartphone,samsung,00000083-8816-4d58-a9b8-f52f54186edc,8,1
10,2019-10-06 11:26:39 UTC,view,1004871,electronics.smartphone,samsung,00000083-8816-4d58-a9b8-f52f54186edc,9,1
11,2019-10-06 11:26:53 UTC,view,1004751,electronics.smartphone,samsung,00000083-8816-4d58-a9b8-f52f54186edc,10,1
12,2019-10-06 11:27:05 UTC,view,1004653,electronics.smartphone,samsung,00000083-8816-4d58-a9b8-f52f54186edc,11,1
13,2019-10-06 11:27:24 UTC,view,1005015,electronics.smartphone,samsung,00000083-8816-4d58-a9b8-f52f54186edc,12,1
14,2019-10-06 11:28:05 UTC,view,1004871,electronics.smartphone,samsung,00000083-8816-4d58-a9b8-f52f54186edc,9,1
15,2019-10-06 11:28:34 UTC,view,1003527,electronics.smartphone,xiaomi,00000083-8816-4d58-a9b8-f52f54186edc,13,1
16,2019-10-06 11:28:45 UTC,view,1004871,electronics.smartphone,samsung,00000083-8816-4d58-a9b8-f52f54186edc,9,1


In [17]:
import random

random.shuffle(sessions)
split = len(sessions) * 0.8
train = sessions[:int(split)]
test = sessions[int(split):]
print('No. train sessions: ', len(train))
print('No. test sessions: ', len(test))

No. train sessions:  237531
No. test sessions:  59383


In [18]:
def jaccard(list1, list2):
    intersection = len(list(set(list1).intersection(list2)))
    union = (len(list1) + len(list2)) - intersection
    return float(intersection) / union

In [19]:
actual_session = test[3]
target = actual_session[1][0:-1]
print(actual_session)
print(target)
subset.loc[subset.sessionId==actual_session[0]]

(np.int64(64833), [1796, 1796, 1315, 1315, 1314, 1315])
[1796, 1796, 1315, 1315, 1314]


,event_time,event_type,product_id,category_code,brand,user_session,itemId,sessionId
269551,2019-10-06 09:23:00 UTC,view,1004669,electronics.smartphone,xiaomi,02997ee2-578e-4955-b9d7-e83a9a6b3f0d,1796,64833
269552,2019-10-06 09:23:36 UTC,view,1004669,electronics.smartphone,xiaomi,02997ee2-578e-4955-b9d7-e83a9a6b3f0d,1796,64833
269553,2019-10-06 09:24:01 UTC,view,1004891,electronics.smartphone,zte,02997ee2-578e-4955-b9d7-e83a9a6b3f0d,1315,64833
269554,2019-10-06 09:31:52 UTC,view,1004891,electronics.smartphone,zte,02997ee2-578e-4955-b9d7-e83a9a6b3f0d,1315,64833
269555,2019-10-06 09:32:16 UTC,view,1004892,electronics.smartphone,zte,02997ee2-578e-4955-b9d7-e83a9a6b3f0d,1314,64833
269556,2019-10-06 09:35:13 UTC,view,1004891,electronics.smartphone,zte,02997ee2-578e-4955-b9d7-e83a9a6b3f0d,1315,64833


In [20]:
def compute_score(train, target, itemId):
    candidate_sessions = []
    for s in range(len(train)):
        if itemId in train[s][1]:
            candidate_sessions.append(train[s][1])
    
    score = 0
    for n in range(len(candidate_sessions)):
        score += jaccard(candidate_sessions[n], target)
    
    return score
    
compute_score(train=train, target=target, itemId=931)

0.02

In [21]:
categories = subset.loc[subset.sessionId==actual_session[0]]['category_code'].unique().tolist()
candidate_items = subset.loc[subset.category_code.isin(categories)]['itemId'].unique().tolist()
candidate_items

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 27,
 28,
 29,
 30,
 31,
 35,
 36,
 37,
 38,
 43,
 44,
 52,
 53,
 56,
 59,
 61,
 71,
 72,
 73,
 74,
 75,
 79,
 81,
 82,
 90,
 91,
 92,
 94,
 95,
 97,
 98,
 113,
 114,
 116,
 134,
 135,
 136,
 143,
 144,
 146,
 147,
 153,
 159,
 170,
 177,
 178,
 179,
 180,
 191,
 200,
 201,
 210,
 211,
 212,
 213,
 214,
 215,
 216,
 217,
 218,
 219,
 220,
 221,
 222,
 239,
 241,
 246,
 248,
 250,
 256,
 257,
 258,
 259,
 260,
 274,
 275,
 276,
 284,
 285,
 287,
 291,
 292,
 296,
 297,
 298,
 299,
 314,
 315,
 318,
 320,
 321,
 322,
 323,
 337,
 338,
 339,
 340,
 341,
 342,
 343,
 344,
 352,
 357,
 359,
 361,
 362,
 367,
 385,
 386,
 387,
 399,
 407,
 423,
 424,
 425,
 426,
 427,
 428,
 429,
 431,
 458,
 459,
 460,
 471,
 479,
 483,
 484,
 485,
 486,
 487,
 493,
 504,
 505,
 506,
 514,
 515,
 516,
 517,
 520,
 528,
 529,
 530,
 548,
 551,
 562,
 574,
 575,
 576,
 577,
 598,
 599,
 605,
 625,
 630,
 638,
 641,


In [22]:
ranking = []
for i in range(len(candidate_items)):
    ranking.append((compute_score(train, target, candidate_items[i]), candidate_items[i]))

ranking.sort()
ranking.reverse()
print(ranking[0:10])

[(24.579714005508116, 1796), (19.06053211447194, 1315), (9.376050168356223, 1314), (5.94402081278592, 479), (5.881407423319004, 274), (4.638210086607684, 341), (4.58377365041171, 598), (3.657377473857068, 248), (3.5472457832323485, 340), (3.036342041973953, 2537)]


In [25]:
subset.loc[subset.itemId==1046]

,event_time,event_type,product_id,category_code,brand,user_session,itemId,sessionId
2519,2019-10-12 18:16:54 UTC,view,5100719,electronics.clocks,samsung,000729f8-7f6c-43f6-b1dd-aab45a29f5c1,1046,652
5660,2019-10-01 13:27:59 UTC,view,5100719,electronics.clocks,samsung,000fdfe4-e1f0-4a93-9c22-f04066ad895e,1046,1461
5705,2019-10-01 10:07:25 UTC,view,5100719,electronics.clocks,samsung,000ff41f-8d94-449e-86fc-25e0957da685,1046,1470
5706,2019-10-01 10:08:04 UTC,view,5100719,electronics.clocks,samsung,000ff41f-8d94-449e-86fc-25e0957da685,1046,1470
6330,2019-10-12 19:48:00 UTC,view,5100719,electronics.clocks,samsung,00118404-d57f-4480-b917-df839bc6a188,1046,1642
...,...,...,...,...,...,...,...,...
1998579,2019-10-31 03:54:14 UTC,view,5100719,electronics.clocks,samsung,1347912f-0cb1-4086-b174-d7f10f275ffa,1046,483132
1998631,2019-10-11 16:58:51 UTC,view,5100719,electronics.clocks,samsung,1347c04a-6cfd-4cca-925e-60cead2e1931,1046,483151
1998632,2019-10-11 16:59:10 UTC,view,5100719,electronics.clocks,samsung,1347c04a-6cfd-4cca-925e-60cead2e1931,1046,483151
1998633,2019-10-11 17:00:34 UTC,view,5100719,electronics.clocks,samsung,1347c04a-6cfd-4cca-925e-60cead2e1931,1046,483151
